<a href="https://colab.research.google.com/github/hyejeong1111/Data-Analysis-/blob/main/Data_Analysis_Practice_Using_Shopping_Mall_Customer_Data_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 목적

- 일반 쇼핑몰에서 발생한 정보(고객, 상품, 주문정보)을 이용하여 기본적은 분석을 진행
- 이를 통해서 pandas를 보다 능숙하게 사용하는것을 목표
- 데이터
  - customer_master.csv(고객정보)
  - item_master.csv(상품정보)
  - transaction_1/transaction_2.csv(매출/주문정보)
  - transaction_detail_1/transaction_detail_2.csv(주문상세정보)

In [6]:
import numpy as np
import pandas as pd

# 데이터를 읽어서 df로 모두 구성

In [7]:
customer_master = pd.read_csv('/content/drive/MyDrive/res/customer_master.csv')
customer_master.head(2)

,customer_id,customer_name,registration_date,email,gender,age,birth,pref
0,IK152942,김서준,2019-01-01 0:25,hirata_yuujirou@example.com,M,29,1990-06-10,대전광역시
1,TS808488,김예준,2019-01-01 1:13,tamura_shiori@example.com,F,33,1986-05-20,인천광역시


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
item_master = pd.read_csv('/content/drive/MyDrive/res/item_master.csv')
item_master.head(2)

,item_id,item_name,item_price
0,S001,PC-A,50000
1,S002,PC-B,85000


In [9]:
transaction_1 = pd.read_csv('/content/drive/MyDrive/res/transaction_1.csv')
transaction_1.head(2)

,transaction_id,price,payment_date,customer_id
0,T0000000113,210000,2019-02-01 01:36:57,PL563502
1,T0000000114,50000,2019-02-01 01:37:23,HD678019


In [10]:
transaction_2 = pd.read_csv('/content/drive/MyDrive/res/transaction_2.csv')
transaction_2.head(2)

,transaction_id,price,payment_date,customer_id
0,T0000005113,295000,2019-06-15 07:20:27,TS169261
1,T0000005114,50000,2019-06-15 07:35:47,HI599892


In [11]:
transaction_detail_1 = pd.read_csv('/content/drive/MyDrive/res/transaction_detail_1.csv')
transaction_detail_1.head(2)

,detail_id,transaction_id,item_id,quantity
0,0,T0000000113,S005,1
1,1,T0000000114,S001,1


In [13]:
transaction_detail_2 = pd.read_csv('/content/drive/MyDrive/res/transaction_detail_2.csv')
transaction_detail_2.head(2)

,detail_id,transaction_id,item_id,quantity
0,5000,T0000004870,S002,3
1,5001,T0000004871,S003,1


# 데이터 통합

- 데이터가 너무 커서 분리되어 있는데이터는 단순 합치기 진행
- 데이터끼리 연결고리가 있는 데이터는 머지수행

In [14]:
transaction_1.shape, transaction_2.shape
# 데이터는 원래 6786개 였는데, 5천대 단위로 끊어서 제공되었다
# 이 데이터를 원대로 한몸으로 만들어야한다 => 구조가 동일하다 => concat

((5000, 4), (1786, 4))

In [15]:
transaction_detail_1.shape, transaction_detail_2.shape
# 상세 정보도 7144개였는데 5천단위로 나눠져 있어서 통합해야 한다
# transaction과 transaction_detail의 총개수가 다르다 (이유는 차차 확인하는것으로 체크)

((5000, 4), (2144, 4))

In [16]:
# transaction_1 + transaction_2 => transaction
transaction = pd.concat( [transaction_1, transaction_2], ignore_index=True )
transaction.shape

(6786, 4)

In [17]:
# 결합된 지점 확인
transaction.iloc[ 5000-2:5000+2 , : ]
# 합쳐진 구간을 살펴보니 인덱스가 4999->5000으로 가지 않고 다시 원래값 0으로 간다
# 차후 인덱스를 조정해야 한다 => reset_index() or ignore_index=True 대체

,transaction_id,price,payment_date,customer_id
4998,T0000005111,210000,2019-06-15 04:14:06,IK074758
4999,T0000005112,50000,2019-06-15 04:42:38,HD444151
5000,T0000005113,295000,2019-06-15 07:20:27,TS169261
5001,T0000005114,50000,2019-06-15 07:35:47,HI599892


In [18]:
# 상세 정보 합치기
transaction_detail = pd.concat( [transaction_detail_1, transaction_detail_2], ignore_index=True )
transaction_detail.shape

(7144, 4)

In [19]:
transaction_detail.iloc[ 5000-2:5000+2 , : ]
# ignore_index 이 옵션을 사용하니까, 인덱스가 자연스럽게 연결되었다

,detail_id,transaction_id,item_id,quantity
4998,4998,T0000004868,S005,1
4999,4999,T0000004869,S003,1
5000,5000,T0000004870,S002,3
5001,5001,T0000004871,S003,1


In [20]:
# merge() 작업
# transaction_detail, transaction
transaction_detail.head(1)

,detail_id,transaction_id,item_id,quantity
0,0,T0000000113,S005,1


In [21]:
transaction.head(1)

,transaction_id,price,payment_date,customer_id
0,T0000000113,210000,2019-02-01 01:36:57,PL563502


In [22]:
# transaction_id : 공통의 컬럼
# transaction에서는 price를 제외하고 합병시 사용하겟다
# 왼쪽에 transaction_detail을 배치한 이유는 데이터수가 더 많아서 이를 기준으로 합친후
# 검토하기 위함
transaction_joined = pd.merge( transaction_detail, transaction[['transaction_id','payment_date','customer_id']],
                              on='transaction_id', how='left')

print( transaction_joined.shape )
transaction_joined.head(2)
# 왼족 df를 기준으로 7144개 데이터가 확정되었고, 컬럼은 기존 4개 + 3개(이중 1개가 공통)=> 4+2개가됨

(7144, 6)


,detail_id,transaction_id,item_id,quantity,payment_date,customer_id
0,0,T0000000113,S005,1,2019-02-01 01:36:57,PL563502
1,1,T0000000114,S001,1,2019-02-01 01:37:23,HD678019


### 2022-05-15 수업진행


In [23]:
#customer_master, item_master, transaction_joined
customer_master.shape, item_master.shape, transaction_joined.shape

((5000, 8), (5, 3), (7144, 6))

In [24]:
transaction_joined.head(1)

,detail_id,transaction_id,item_id,quantity,payment_date,customer_id
0,0,T0000000113,S005,1,2019-02-01 01:36:57,PL563502


In [25]:
# transaction_joined, customer_master => merge , on=customer_id, how='left'


#=> 거래정보에 고객정보가 누락된 것이 있는지
join_data = pd.merge( transaction_joined, customer_master, on='customer_id', how='left')
join_data.shape
display( join_data.head(1) )

,detail_id,transaction_id,item_id,quantity,payment_date,customer_id,customer_name,registration_date,email,gender,age,birth,pref
0,0,T0000000113,S005,1,2019-02-01 01:36:57,PL563502,김태경,2019-01-07 14:34,imoto_yoshimasa@example.com,M,30,1989-07-15,대전광역시


In [26]:
#join_data, item_master 를 최종 merge()에서 최종 1개의 df 로 구성완료
join_data=pd.merge(join_data, customer_master, on='customer_id', how='left')
print(join_data.shape)
join_data.head(1)
# join_data에 1개의 정보에는 트렌젝션,아이템,고객정보가 다 연결되어 있다

(7144, 20)


,detail_id,transaction_id,item_id,quantity,payment_date,customer_id,customer_name_x,registration_date_x,email_x,gender_x,age_x,birth_x,pref_x,customer_name_y,registration_date_y,email_y,gender_y,age_y,birth_y,pref_y
0,0,T0000000113,S005,1,2019-02-01 01:36:57,PL563502,김태경,2019-01-07 14:34,imoto_yoshimasa@example.com,M,30,1989-07-15,대전광역시,김태경,2019-01-07 14:34,imoto_yoshimasa@example.com,M,30,1989-07-15,대전광역시


##필요한 데이터를 기준 칼럼 조정(추출)추가

In [27]:
# 수량 => quantity
join_data['quantity']

0       1
1       1
2       1
3       1
4       2
       ..
7139    1
7140    1
7141    2
7142    1
7143    1
Name: quantity, Length: 7144, dtype: int64

In [28]:
# 컬럼은 자동으로 df의 맴버변수가 된다
join_data.item_price

AttributeError: ignored

In [29]:
#컬럼 추가 -> 파생변수 생성 :(7144,15)-> (7144,16)
#변수{컬럼명}=데이터(데이터는 df의 개수와 동일->7144)
joined_data['price']=join_data['quantity']+join_data.item_price    #수량 +아이템가격
joined_data.shape

AttributeError: ignored

In [ ]:
join_data.head(1)

NameError: ignored

# 필요한 데이터를 기준 컬럼 조정(추출)

# 데이터가 정확한 것인가? 검증과정

- 데이터를 신뢰할수 있는가?

In [ ]:
#join_data에서 price의 데이터 총합이 transaction에서 price의 총합과 일치하는가 ?
join_data.shape, transaction.shape

((7144, 20), (6786, 4))

In [1]:
#합산 후 비교
join_data['price'].sum()==transaction.price.sum()
#True => 데이터는 무결하다. 이상없다(매출에 관해서는)

NameError: ignored

# EDA 진행

- 각종 통계 지표 점검

In [ ]:
#데이터 자체에 결측 정보는 없다.
join_data.isnull().sum()

NameError: ignored

In [ ]:
#전체 칼럼(feature, 특징) 16-> 5개만 노출됨(수치만 표현된다.)
join_data.describe()
#기초 통계 요약을 통해서 이 데이터를 이해하고, 제공한 회사의 상황도 이해
#=> 이 데이터에 대한 인사이트를 구축해 나간다

,detail_id,quantity,age_x,age_y
count,7144.000000,7144.000000,7144.000000,7144.000000
mean,3571.500000,1.199888,50.265677,50.265677
std,2062.439494,0.513647,17.190314,17.190314
min,0.000000,1.000000,20.000000,20.000000
25%,1785.750000,1.000000,36.000000,36.000000
50%,3571.500000,1.000000,50.000000,50.000000
75%,5357.250000,1.000000,65.000000,65.000000
max,7143.000000,4.000000,80.000000,80.000000


In [ ]:
join_data.colunm

NameError: ignored

In [ ]:
#구매일 최초, 마지막
join_data['payment_data'].min(), join_data['payment_data'].max()
#데이터는 19년 2월 1일부터 7월 11일까지 데이터이다


KeyError: ignored

# 월별 데이터 통계량 집계

- groupby 사용 예상

In [ ]:
join_data.head(1)
#현재 날짜 데이터는 년-월-일 시:분:초로 되어 있다 => 년월을 추출해서 새로운 칼럼을 생성
#=> 년월을 추출해서 새로운 칼럼을 생성, 월별 집계 데이터를 만들 수 있다.
#이 정보는 문자열로 취급해서 처리하거나, 시간타입을 처리해서 처리할 수 있다.
#각 컬럼의 타입 확인

,detail_id,transaction_id,item_id,quantity,payment_date,customer_id,customer_name_x,registration_date_x,email_x,gender_x,age_x,birth_x,pref_x,customer_name_y,registration_date_y,email_y,gender_y,age_y,birth_y,pref_y
0,0,T0000000113,S005,1,2019-02-01 01:36:57,PL563502,김태경,2019-01-07 14:34,imoto_yoshimasa@example.com,M,30,1989-07-15,대전광역시,김태경,2019-01-07 14:34,imoto_yoshimasa@example.com,M,30,1989-07-15,대전광역시


In [ ]:
#2019-02-01 01:36:57 => '2019-02-01'=> '201902'
'2019-02-01 01:36:57'[:7].replace('-','')


'201902'

In [ ]:
join_data.dtypes

detail_id               int64
transaction_id         object
item_id                object
quantity                int64
payment_date           object
customer_id            object
customer_name_x        object
registration_date_x    object
email_x                object
gender_x               object
age_x                   int64
birth_x                object
pref_x                 object
customer_name_y        object
registration_date_y    object
email_y                object
gender_y               object
age_y                   int64
birth_y                object
pref_y                 object
dtype: object

In [ ]:
#방법1: 문자열로 처리한다면
def convert(x):
  #함수 내부에 수행문이 1줄이다 => lambda 으로 대체 가능=> 더 빠르다. 휘발설.
  return x [:7].replace('-','')
  pass
#join_data.payment_data.apply(convert)
#데이터를 하나씩 꺼내서 람다함수에 넣어서 전처리를 리턴=> 전처리 후 리턴
#=> 전체 데이터가 일괄 가공되서 series로 나온다.
tmp =join_data.payment_data.apply(lambda x: x [:7].replace('-',''))
print(tmp)
join_data


AttributeError: ignored

In [ ]:
#방법2=>
#시간/날짜 타입은 object로 보이는 경우 실제 시간항으로 변형하여 정보 추출
#시간 정보를 조작할 수 있는 타입으로 변경해서 새로 다시 대입처리
#기존데이터가 있으면 수정, 없으면 추가
join_data['payment_date']=pd.to_datetime(join_data['payment_date'])

NameError: ignored

In [ ]:
join_data['payment_date'].dt.strftime ('%Y%m')

NameError: ignored

In [ ]:

#칼럼 추가
#join_data['payment_month']=tmp
join_data.head(1)

NameError: ignored

In [ ]:
join_data.head(1)
#(월별 단위로 아이템별로) 판매 수량/총매출을 출력하시오.


NameError: ignored

In [ ]:
join_data.groupby(['payment_month','item_name']).sum()[['price','quantity']]
#그룹 대상이 2개가 되니까, 인덱스가 level이 발생하였다.

NameError: ignored

In [ ]:
#피벗테이블(범주형데이터(구분용데이터)가 기존) 활용 데이터를 살펴본다.
pd.pivat_table(join_data,index='payment_month',columns=item_name,values='price', aggfunc='sum')
#피벗을 수행하면 숫자만 있는 칼럼만 포함된다.
#aggfunc 값으로 ['sum']=> 칼럼(혹은 인덱스) depth가 늘어난다.

AttributeError: ignored

# 월별 상품별 데이터를 집계

In [ ]:
#월(별로)단위로 매출 합계량은?=> groupby
#unique()=> 특정 칼럼에 존재하는 데이터들 중 중복을 제거하고 출력
#groupby를 적용하는 칼럼의 데이터는 통상적으로 범주형 데이터일 가능성이 크다
#대부분은 셀 수 있다(뉘앙스)
join_data['payment_month'].unique()


NameError: ignored

In [ ]:
#월별로 매출 합계
join_data.groupby('payment_month').sum()['price']

NameError: ignored

# 월별 데이터를 기준으로 시각화

- 차트를 그려서 이 데이터를 설명
- 시각적 분석법
  - 시각화 도구

  - x축은 201902, 201903,....201907
  - y축은 PC-A or PC-B,..등의 개별 데이터

가격데이터
 - 선형그래프로 그리시오 (plt.plot())
 - 차트안에 그래프는 총 5개가 나온다
 (범례표기해야함)

In [ ]:

from matplotlib import pyplot as plt
import matplotlib

In [ ]:
%matplotlib inline

In [ ]:
# 1.데이터 준비
x = tmp.index
x

NameError: ignored

In [ ]:
for col in tmp.columns:
  y = tmp[col]
  print( y )
  break

NameError: ignored

In [ ]:
# 2.그리기
plt.figure()
for col in tmp.columns:
  plt.plot( x, tmp[col], label=col ) # col => 'PC-A', PC-B,... PC-E
plt.legend()
plt.show()

NameError: ignored

<Figure size 432x288 with 0 Axes>